# Training a Baseline Model: GPT-2 fine-tuned for NLI using all of SNLI train data

### 1. Imports and Global Settings

In [1]:
from datasets import load_dataset, disable_caching
from tqdm.notebook import tqdm
from transformers import GPT2ForSequenceClassification, GPT2TokenizerFast, DataCollatorWithPadding, set_seed
import torch
from torch.nn.functional import one_hot
from utils_ import tokenize, train_classifier
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
set_seed(42)
disable_caching()

### 2. Pre-Processing
- Get SNLI Dataset (Train fold)
- Remove instances without gold standard labels, i.e., label = -1
- One-hot encoding for labels
- Tokenise data

In [2]:
train = load_dataset('snli', split = 'train')
train = train.filter(lambda x: x['label'] != -1).map( \
    lambda x: {'label': one_hot(torch.tensor(x['label']), 3).type(torch.float32).numpy()}, \
    batched = True)

Reusing dataset snli (/home/shana92/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

In [3]:
# set up tokeniser
# padding to left because GPT2 uses last token for prediction
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", padding_side = 'left', \
                                              padding = True, truncation = True)
tokenizer.pad_token = tokenizer.eos_token # pad with 'eos' token

In [4]:
# tokenize data
train = train.map(lambda x: tokenize(tokenizer, x['premise'] + '|' + x['hypothesis']))
len_bef_exclusion = len(train)

# exclude instances with > 128 tokens
train = train.filter(lambda x: x['exclude'] == False)
len_aft_exclusion = len(train)

# print message if instances were in fact excluded
if len_bef_exclusion - len_aft_exclusion > 0:
    
    print(f'{len_bef_exclusion - len_aft_exclusion} ' + \
          f'({(en_bef_exclusion/len_aft_exclusion - 1):>2f}%) sequences excluded')

  0%|          | 0/549367 [00:00<?, ?ex/s]

  0%|          | 0/550 [00:00<?, ?ba/s]

In [5]:
# keep only needed columns, set data format to PyTorch
train.set_format(type = 'torch', columns = ['label', 'input_ids', 'attention_mask'])

### 3. Model

In [6]:
# set up data collator - https://huggingface.co/docs/transformers/main_classes/data_collator
# this is a (callable) helper object that sends batches of data to the model
data_collator = DataCollatorWithPadding(tokenizer, padding = 'max_length', \
                                         return_tensors = 'pt', max_length = 128)

In [7]:
# hyper-parameters for model training - set to match Le et al. (2020) - https://arxiv.org/abs/2002.04108
batch_size = 92
lr = 1e-5

In [8]:
# set up a dataloader (batch generator)
dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, \
                                 shuffle=True, collate_fn=data_collator) # batch size constrained by GPU memory

In [9]:
# instantiate GPT2 classifier model
model = GPT2ForSequenceClassification.from_pretrained("gpt2", 
                                  num_labels=3,
                                  problem_type="multi_label_classification")
model.config.pad_token_id = model.config.eos_token_id # specify pad_token used by tokenizer

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# set up optimizer (loss function in-built)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [11]:
# Train and Save model
torch.save(train_classifier(model, dataloader, optimizer, device), 'baseline_unfiltered.pth')

  0%|          | 0/5972 [00:00<?, ?it/s]

loss: 1.863892  [    0/549367]
loss: 0.607553  [54924/549367]
loss: 0.528493  [109848/549367]
loss: 0.506676  [164772/549367]
loss: 0.478043  [219696/549367]
loss: 0.399283  [274620/549367]
loss: 0.419539  [329544/549367]
loss: 0.373558  [384468/549367]
loss: 0.329311  [439392/549367]
loss: 0.413944  [494316/549367]
loss: 0.437282  [549240/549367]
Epoch average loss: 0.44208070635795593


  0%|          | 0/5972 [00:00<?, ?it/s]

loss: 0.267417  [    0/549367]
loss: 0.261624  [54924/549367]
loss: 0.352897  [109848/549367]
loss: 0.360383  [164772/549367]
loss: 0.270687  [219696/549367]
loss: 0.397392  [274620/549367]
loss: 0.249163  [329544/549367]
loss: 0.267761  [384468/549367]
loss: 0.311435  [439392/549367]
loss: 0.253674  [494316/549367]
loss: 0.280305  [549240/549367]
Epoch average loss: 0.3083632290363312


  0%|          | 0/5972 [00:00<?, ?it/s]

loss: 0.258525  [    0/549367]
loss: 0.323325  [54924/549367]
loss: 0.350854  [109848/549367]
loss: 0.217245  [164772/549367]
loss: 0.292087  [219696/549367]
loss: 0.273418  [274620/549367]
loss: 0.351640  [329544/549367]
loss: 0.221741  [384468/549367]
loss: 0.242559  [439392/549367]
loss: 0.307192  [494316/549367]
loss: 0.280019  [549240/549367]
Epoch average loss: 0.27260008454322815
Done!
